# NO LONELY COLOR DETECTOR: Two color version
In this file we have three sections:  
   1. **Main functions needed:** Cell that includes the main functions and imports that will be needed in this notebook.  
   2. **Going through all the graphs of a file:** Cell that can be used to go through all the files saved in a g6 file. Before using this cell make sure that the variable 'folderPath' is defined as needed.  
   3. **Checking an specific graph:** Cell that can be used to check an specific graph. To do so, we have to make sure to update the variable 'g6_string' with the graph6 format of the graph that we want to check.

# Main functions needed

In [ ]:
import os
import time
import supportFunctions as sf
from twoColorsSolutionChecker import isSolutionCorrect

def checkAllTwoColorings(G):
    """
    Functiong that calls the backtracking funcion that goes through all the colorings in the
    search of a no lonely coloring.
    
    PARAMETERS:
        G: Graph
    
    RETURNS:
        True if it finds a no lonely coloring in G.
        False if not
    """
    return checkTwoColorings_backtrack(G.copy(), G.copy(), Graph(), G.edges()[0], 1, False, set(), 0)


def checkTwoColorings_backtrack(G, G_aux, G_coloredEdges, edge, color, colorIsOkey, statesChecked, lenMonochromaticCycle):
    """
    Backtracking function that searchs a no lonely coloring only using 2 colors in G.
    
    PARAMETERS:
        G:                     Graph that we want to color
        G_aux:                 Graph that contains the edges that are not colored with "color"
        G_coloredEdges:        Graph that contains the colored edges
        edge:                  Edge that is going to be colored next
        color:                 Color that we will use to color "edge" (in this case it will always be 1)
        colorIsOkay:           Boolean that tells us that, once we color "edge", if the edges colored
                                 with "color" satisfy the needed conditions to be a no lonely color
        statesChecked:         Set that contains the colorings that we have already discarded.
        lenMonochromaticCycle: If we find a monochromatic cycle, it contains the length. If not it is 0.
    
    RETURNS:
        True if it finds a no lonely coloring in G.
        False if not
    """ 
    u, v, label = edge[0], edge[1], edge[2]

    # If the edge is uncolored
    if label == 0:
        # We color the edge and update all the graphs
        G.set_edge_label(u, v, color)
        G_aux.delete_edge(u, v)
        G_coloredEdges.add_edge(u, v, color)
        
        # DEFINING edges_missing --> WE WANT TO KNOW WHICH EDGES SHOULD BE COLORED NEXT
        # We search for lonely uncolored edges in cycles
        checkNonColoredCycles, uncoloredEdges = sf.searchOfLonelyEdgesByColor(G, 0)
        
        if not checkNonColoredCycles:
            # uncoloredEdges contains a lonely uncolored edge in a cycle
            edges_missing = uncoloredEdges
        else:
            # Else we search for a lonely edge with color "color"
            checkCyclesColor, uncoloredEdges = sf.searchOfLonelyEdgesByColor(G, color)
            
            if not checkCyclesColor:
                # uncoloredEdges contains the edges of the cycle that contains a lonely edge
                edges_missing = uncoloredEdges
            else:
                # If we have not found a lonely edge of color "color"...
                if colorIsOkey:
                    # We check if the solution is correct and the lengths of the monochromatic cycles
                    if isSolutionCorrect(G, 0, color) and sf.searchMonochromaticCyclesByColor(G,0):
                        return True
                    
                # If we have no candidates, we just try to color all the uncolored edges
                edges_missing = G_aux.edges()
        
        # WE EChecking an specific graphXPLORE THE CANDIDATES TO COLOR
        for edge_to_color in edges_missing:
            u_to_color, v_to_color, label_to_color = edge_to_color[0], edge_to_color[1], edge_to_color[2]

            if label_to_color == 0:                
                G_coloredEdges.add_edge(u_to_color, v_to_color, color)
                
                # If we have not discarded this state before
                if tuple(G_coloredEdges.edges()) not in statesChecked:
                    # If the vertices do not have three adjacent edges of color "color"
                    if sf.isDegreeColorGood(G_coloredEdges, edge_to_color, color):
                        lenMonochromaticCycle_currentEdge = sf.searchMonochromaticCyclesInEdge(G_coloredEdges,
                                                                                        edge_to_color, color)
                        
                        # If the monochromatic cycles have the same length
                        if (
                            lenMonochromaticCycle_currentEdge == 0
                            or lenMonochromaticCycle == 0
                            or lenMonochromaticCycle_currentEdge == lenMonochromaticCycle
                        ):
                            if lenMonochromaticCycle == 0:
                                # We have not found a previous monochromatic cycle. So, we save the new value
                                lenMonochromaticCycle = lenMonochromaticCycle_currentEdge
                            
                            # We update G_aux
                            G_aux.delete_edge(u_to_color, v_to_color)

                            # If G_aux is disconnected and G_colored edges has both endpoints
                            # of each edge of color "color" in different components
                            if (
                                not G_aux.is_connected()
                                and sf.areEndpointsInDifferentComponents(G_coloredEdges,
                                                                      G_aux.connected_components(), color)
                            ):
                                # The edges of color "color" satisfy the necessary conditions
                                # --> colorIsOkay = True
                                if checkTwoColorings_backtrack(G, G_aux, G_coloredEdges, edge_to_color,
                                                            color, True, statesChecked, lenMonochromaticCycle):
                                    return True

                            # If not, we continue considering that the color is not okay
                            # --> colorIsOkay = False
                            if checkTwoColorings_backtrack(G, G_aux, G_coloredEdges, edge_to_color,
                                                        color, False, statesChecked, lenMonochromaticCycle):
                                return True

                            # We restore G_aux
                            G_aux.add_edge(u_to_color, v_to_color, 0)
                
                # We restore G_coloredEdges
                G_coloredEdges.delete_edge(u_to_color, v_to_color)
        
        # NONE OF THE edges_missing CAN BE COLORED WITH color
        # To save memory, we do not save the cases that are the only option after a previous state
        if len(edges_missing) != 1: statesChecked.add(tuple(G_coloredEdges.edges()))
        
        # We restore the graphs
        G.set_edge_label(u, v, 0)
        G_aux.add_edge(u, v, 0)
        G_coloredEdges.delete_edge(u, v)

    return False


def isNoLonelyColor_2colors(G):
    """
    Main functions that checks if a given graph has a no lonely coloring using only 2 colors.
    
    PARAMETERS:
        G: Graph
    
    RETURNS:
        True if G is a no lonely coloring.
        False if the graph is not a no lonely coloring using 2 colors.
    """
    return checkAllTwoColorings(G)

# Going through all the graphs of a file

In [ ]:
# WRITE THE FOLDER THAT YOU WANT TO USE
folderPath = './'

# We go through the files in the folder "folderPath"
for file in os.listdir(folderPath):
    if file.endswith('.g6') or file.endswith('.graph6'):
        filePath = os.path.join(folderPath, file)

        check = input(f"Do you want to open the file: {file}? (y/n)")
        
        if check == 'y':            
            start_global = time.time()            
            f = open(filePath, "r")
            lines = f.readlines()

            for i, line in enumerate(lines):
                start = time.time()
                g6_string = line

                try:
                    G = Graph(g6_string)
                except Exception as e:
                    print(f"Exception: {e}")
                    print(f"Graph: {g6_string}")
                    continue

                # We set the labels of the edges as 0
                for u, v, _ in G.edges():
                    G.set_edge_label(u, v, 0)

                if not isNoLonelyColor_2colors(G):
                    # If we have not found a no lonely coloring...
                    print(f"Graph {i} in {file}: FALSE")
                    print(g6_string)

                    # Draw the SageMath graph
                    G.show()

                end = time.time()
                print(f"{g6_string} checked in {end-start}s")

            end_global = time.time()
            print(f"{file} checked in {end_global-start_global}s")

# Checking an specific graph

In [ ]:
# WRITE GRAPH IN FORMAT g6
g6_string = r"KsP@PGWCOH?R"

try:
    G = Graph(g6_string)
except:
    print("ERROR: Introduce a valid graph in g6 format.")
    G = Graph()

for u, v, _ in G.edges():  # We set the edges of the graph as 0
    G.set_edge_label(u, v, 0)

start = time.time()
if not isNoLonelyColor_2colors(G):
    print("No lonely color graph found.")

    # Draw the SageMath graph
    G.show()

end = time.time()
print(f"{g6_string} checked in {end-start}s")